In [43]:
import pygame_2d
import numpy as np
from const import*

screen = np.zeros((720, 1280, 3), dtype = np.uint8)



In [44]:

high, low = 1,0

alpha = 0, 2*math.pi    # góc quay 
rVelo = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO  # vận tốc quay 
fwVelo = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO    # vận tốc tiến



In [45]:

gamma = 0.99
alpha = 0.1
epsilon = 1
epsilon_decay = epsilon / 4000
n_epsilondes = 10000



In [46]:
random.seed(13)
lidars = np.random.randint(0, 360, size= 360)

bin_lidarsignal = np.linspace(0, 360, num= LENGTH_LIDARSIGNAL, endpoint= True)
bin_lidarsignal = np.delete(bin_lidarsignal, 0)
lidars = np.digitize(lidars, bin_lidarsignal)

lidars[lidars == 3] = INT_INFINITY
const_lidars = lidars
temp_lidars = lidars


NameError: name 'LENGTH_LIDARSIGNAL' is not defined

In [ ]:
bin_lidarspace = np.array([45, 135, 180])
lidars = np.array_split(temp_lidars, bin_lidarspace)

# dis_lidars = {"0": (np.amin(lidars[0]), lidars[0]),
#               "1": (np.amin(lidars[1]), lidars[1]),
#               "2": (np.amin(lidars[2]), lidars[2]),
#               "3": INFINITY}

section_lidars_min = [np.amin(lidars[0]), np.amin(lidars[1]), np.amin(lidars[2]), INT_INFINITY]
lidarStateVector = np.array(section_lidars_min)
lidarStateVector

array([    0,     0,     0, 99999], dtype=int64)

In [ ]:

high, low = 1, 0
alpha_space = 0, 2*math.pi
fwVelo_space = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO
rVelo_space = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO

lowState = np.array(
    [alpha_space[0], fwVelo_space[0], rVelo_space[0]], dtype=float)
upState = np.array(
    [alpha_space[1], fwVelo_space[1], rVelo_space[1]], dtype=float)

infoState_shape = (ALPHA_SPACE, FWVELO_SPACE, RVELO_SPACE)

bins = []
for i in range(3):
    item = np.linspace(
        lowState[i],
        upState[i],
        num=infoState_shape[i],
        endpoint=False)
    item = np.delete(item, 0)
    bins.append(item)
    print(bins[i])


[0.31415927 0.62831853 0.9424778  1.25663706 1.57079633 1.88495559
 2.19911486 2.51327412 2.82743339 3.14159265 3.45575192 3.76991118
 4.08407045 4.39822972 4.71238898 5.02654825 5.34070751 5.65486678
 5.96902604]
[-50.   0.  50.]
[-0.5  0.   0.5]


In [ ]:
alpha = 0.3993
fwVelo = 23
rVelo = 0.32
s = (alpha, fwVelo, rVelo)
infoStateVector = []
for i in range(3):
    infoStateVector.append(np.digitize(s[i], bins[i]))

infoStateVector = np.array(infoStateVector)
infoStateVector

array([1, 2, 2], dtype=int64)

In [ ]:
np.concatenate((infoStateVector, lidarStateVector))

array([    1,     3,     0,     0,     0,     0, 99999], dtype=int64)

In [ ]:
new_observation_shape = (ALPHA_SPACE, FWVELO_SPACE, RVELO_SPACE, SECTIONS_LIDARSPACE)
q_table = np.zeros(new_observation_shape + (ACTION_SPACE,))
q_table.shape

(20, 4, 4, 4, 6)